In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from eyeq import block

In [ ]:
# Generate some samples...
phase = np.arange(50000)
samples = np.round((((np.random.randn(50000)+ 1j * np.random.randn(50000)) + np.exp(1j * (0.1 * np.arange(50000) * np.pi * 2 + 0.5 * ((np.arange(50000)/1000)**2) * np.pi * 2))))).astype(np.complex64)

In [ ]:
plt.psd(samples);

In [ ]:
# Construct the blocks that we write to our fake block store.
nsamp = block.BLOCK_F32_SAMPLES
blocks = []
for i in range((len(samples)+nsamp-1) // nsamp):
    data = samples[nsamp*i:nsamp*(i+1)]
    bl = block.Block.create(block.BLOCK_TYPE_I16_SAMPLES)
    bl.header.block_id = i
    bl.init_block_data(data.view(np.float32))
    blocks.append(bl)

In [ ]:
class Store:
    def __init__(self, blocks):
        self.blocks = blocks

    def read_block(self, id):
        return self.blocks[id]


In [ ]:
store = Store(blocks)

In [ ]:
plt.psd(store.read_block(2).samples().view(np.complex64));

In [ ]:
STREAM_BUF = 16384

In [ ]:
class StoreReader:
    def __init__(self, store):
        self.block_id = 0
        self.block_offset = 0
        self.store = store

    def seek(self, block_id):
        self.block_id = block_id

    def read(self, n):
        self


In [ ]:
class ArrayStream:
    def __init__(self, array, start=0):
        self.array = array.astype(np.float32)
        self.position = 0
        self.start = start
    
    def read(self, n):
        output = np.zeros(n, dtype=np.float32)
        data_slice = self.array[self.position+self.start:self.position+self.start+n]
        output[:len(data_slice)] = data_slice
        self.position += n
        return output

    def seek(self, n):
        self.position = n

    def tell(self):
        return self.position
        

In [ ]:
class FrequencyShiftStream:
    def __init__(self, backend, relative_freq, phase):
        self.offset = 0
        self.relative_freq = relative_freq
        self.phase = phase
        self.backend = backend
    
    def read(self, n):
        output = np.zeros(n, dtype=np.float32)
        output_count = 0
        while n > 0:
            data = self.backend.read(n)
            output[output_count:output_count+len(data)] = (data.view(np.complex64) * np.exp(1j * 2 * np.pi * np.arange(self.offset, self.offset+len(data)//2) * self.relative_freq + self.phase)).astype(np.complex64).view(np.float32)
            output_count += len(data)
            n -= len(data)
            self.offset += len(data)//2
        return output

    def seek(self, n):
        self.backend.seek(n)
        self.offset = self.backend.tell()


In [ ]:
class ConvolveCRStream:
    def __init__(self, backend, taps):
        self.backend = backend
        self.taps = np.flipud(taps)
        self.overlap = (len(self.taps)-1)*2
        self.buffer = np.zeros(STREAM_BUF, dtype=np.float32)
        self.reset()

    def reset(self):
        self.offset = 0
        self.buffer[:] = 0
        self.data_offset = self.overlap
        self.shift_and_read()
        
    def fill(self):
        while self.data_offset < len(self.buffer):
            to_read = len(self.buffer) - self.data_offset
            data = self.backend.read(to_read)
            self.buffer[self.data_offset:self.data_offset+len(data)] = data
            self.data_offset += len(data)
    
    def shift_and_read(self):
        """Shift tail end to start of buffer and fill with new data"""
        self.buffer[:self.overlap] = self.buffer[-self.overlap:]
        self.data_offset = self.overlap
        self.fill()
        self.offset = 0

    def fir_filter(self):
        """Produce one output sample by filtering the current position in the input stream."""
        if self.offset + len(self.taps) * 2 > len(self.buffer):
            self.shift_and_read()
        input_data = self.buffer[self.offset:self.offset+len(self.taps)*2].view(np.complex64)
        self.offset += 2
        return np.sum(np.multiply(input_data, self.taps))

    def read(self, n):
        """Read filtered data"""
        output = np.zeros(n // 2, dtype=np.complex64)
        for i in range(len(output)):
            output[i] = self.fir_filter()
        return output.view(np.float32)

    def seek(self, n):
        self.backend.seek(n)
        self.reset()
            

In [ ]:
from scipy.signal import firwin
taps = firwin(41, 0.01)

In [ ]:
convolved = np.convolve(samples, taps, 'full')
convolved_and_shifted = np.convolve(samples * np.exp(1j * 2 * np.pi * np.arange(len(samples)) * (-0.09)), taps, 'full')
plt.plot(np.real(convolved_and_shifted)[8000:9000]);
plt.plot(np.imag(convolved_and_shifted)[8000:9000]);

In [ ]:
stream = ArrayStream(samples.astype(np.complex64).view(np.float32))
fs_stream = FrequencyShiftStream(stream, -0.09, 0)
cr_stream = ConvolveCRStream(fs_stream, taps)

In [ ]:
cr_stream.seek(16000)
data = cr_stream.read(2000).view(np.complex64)
plt.plot(np.real(data));
plt.plot(np.imag(data));

In [ ]:
#plt.psd(data * 0);
#plt.psd(convolved * 0);
plt.psd(convolved_and_shifted[8000:9000]);
plt.psd(data);

In [ ]:
input_data = np.random.randn(2000);

In [ ]:
plt.plot(taps);

In [ ]:
plt.plot(input_data);

In [ ]:
input_data = input_data.astype(np.float32)
input_data.tofile('data.bin')

In [ ]:
taps = taps.astype(np.float32)

In [ ]:
taps.astype(np.complex64).tofile('taps.bin')

In [ ]:
np.convolve(input_data.view(np.complex64), taps, 'full').astype(np.complex64).view(np.float32).tofile('complex_convolved.bin')

In [ ]:
plt.plot(np.convolve(input_data.view(np.complex64), taps, 'full').astype(np.complex64).view(np.float32))

In [ ]:
from scipy.signal import firwin, firwin2

In [ ]:
import numpy as np

In [ ]:
lowpass100 = firwin(121, 150000 / 200000.0)
lowpass50 = firwin(121, 86000 / 200000.0)
lowpass25 = firwin(121, 56000 / 200000.0)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.psd(lowpass25, NFFT=16384);

In [ ]:
lowpass25.tolist()